<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>From local Spark MLlib model to cloud with watson-machine-learning-client</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://www-03.ibm.com/services/learning/images/journey_watson_wat_machine_learning_600x280.jpg" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to work with **``watson-machine-learning-client``** library. This notebook introduces commands for getting data, model persistance to Watson Machine Learning repository, model deployment and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3 and Apache® Spark 2.0.

You will use publicly available data "GoSales Transactions for Naive Bayes Model" about anonymous outdoor equipment purchase to predict clients' interests in terms of product line like: golf accessories, camping equipement and others.


## Learning goals

The learning goals of this notebook are:
- Load and explore data
- Prepare data for training and evaluation
- Create spark ml pipeline
- Train and evaluate model
- Persist local model in Watson Machine Learning repository using client library
- Deploy model for online scoring using client library
- Score sample records using client library


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create an Apache® Spark machine learning model](#model)
4.	[Persist created model using api client](#persistance)
5.	[Deploy and score in a Cloud using api client](#scoring)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered). 
-  Make sure that you are using a Spark 2.0 kernel.

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then *read* method. 

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.

In [1]:
!pip install wget --user 

  Running setup.py bdist_wheel for wget ... done
  Stored in directory: /gpfs/fs01/user/s488-d02ef698016a9a-8ef090487f81/.cache/pip/wheels/6d/98/29/61ccc41148f871009126c2e844e26f73eeb25e12cca92228a5
Successfully built wget


In [2]:
import wget
import json

In [3]:
link_to_data = 'https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600'
filename = wget.download(link_to_data)

print(filename)

GoSales_Tx_NaiveBayes.csv


The csv file GoSales_Tx_NaiveBayes.csv is availble on gpfs now. Load the file to Apache® Spark DataFrame using code below.

In [9]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [10]:
df_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



As you can see, the data contains five fields. PRODUCT_LINE field is the one we would like to predict (label).

In [11]:
df_data.show()

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

In [12]:
df_data.count()

60252

As you can see, the data set contains 60252 records.

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [13]:
splitted_data = df_data.randomSplit([0.8, 0.2], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))

Number of training records: 48176
Number of testing records : 12076


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [14]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.

In [15]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

In the following step, create a feature vector by combining all features together.

In [16]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [17]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, indexed labels back to original labels.

In [18]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [19]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [20]:
model_rf = pipeline_rf.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [21]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

print("Accuracy = %g" % accuracy)

Accuracy = 0.585045


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist created model with ``watson-machine-learning-client``

In this section you will learn how to store your model in Watson Machine Learning repository by using common python client.

#### First, you must install client libraries from pypi

In [8]:
!pip install watson-machine-learning-client --user

    100% |████████████████████████████████| 225kB 4.8MB/s eta 0:00:01


Import installed client by running below code.

In [22]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**You can check sphinx documentation for watson-machine-learning-client either by visiting [documentation webpage](https://wml-api-pyclient.mybluemix.net) or running below code.**

In [1]:
from IPython.display import IFrame
IFrame('https://wml-api-pyclient.mybluemix.net', width='100%', height=400)

### 4.1: Work with your instance

Authenticate to Watson Machine Learning service on Bluemix.

**Action**: Put authentication information from your instance of Watson Machine Learning service here.</div>

In [24]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "*****",
  "username": "*****",
  "password": "*****",
  "instance_id": "*****"
}

**Tip**: Credentials can be found on **Service Credentials** tab of service instance created in Bluemix. If you cannot see **instance_id** field in **Serice Credentials** generate new credentials by pressing **New credential (+)** button. 

#### Create API client by running below code.

In [25]:
client = WatsonMachineLearningAPIClient(wml_credentials)

#### Get instance details.

In [26]:
instance_details = client.service_instance.get_details()

print(json.dumps(instance_details, indent=2))

{
  "metadata": {
    "created_at": "2017-07-17T11:30:15.027Z",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b",
    "modified_at": "2017-10-10T10:35:24.828Z",
    "guid": "81eb122f-a170-4efd-86b6-4a8afa16d30b"
  },
  "entity": {
    "region": "us-south",
    "account": {
      "id": "500838470",
      "type": "TRIAL",
      "name": "Lukasz Cmielowski's Account"
    },
    "deployments": {
      "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/deployments"
    },
    "source": "Bluemix",
    "status": "Active",
    "plan_id": "0f2a3c2c-456b-40f3-9b19-726d2740b11c",
    "published_models": {
      "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models"
    },
    "organization_guid": "5a46f2ad-87af-451a-a6ea-684c0bb92ce9",
    "plan": "standard",
    "space_guid": "e6d9f948-d4d4-411b-9ce4-3fbaf243aed6",
    "usage": {
      "

### 4.2: Publish model

#### Publish model in Watson Machine Learning repository on Cloud.

In [27]:
published_model = client.repository.publish(model_rf, "LOCALLY created Product Line Prediction model", meta_props={"authorName":"IBM", "authorEmail":"ibm@ibm.com"}, training_data=train_data)

### 4.3: Get model details

In [28]:
model_details = client.repository.get_details(published_model.uid)
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "created_at": "2017-10-10T11:26:32.270Z",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models/23d93142-c740-4cec-9cd6-18dad8bb9717",
    "modified_at": "2017-10-10T11:26:32.536Z",
    "guid": "23d93142-c740-4cec-9cd6-18dad8bb9717"
  },
  "entity": {
    "label_col": "PRODUCT_LINE",
    "latest_version": {
      "created_at": "2017-10-10T11:26:32.536Z",
      "url": "https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/23d93142-c740-4cec-9cd6-18dad8bb9717/versions/57899b60-5336-448d-81cb-ec140e7e8112",
      "guid": "57899b60-5336-448d-81cb-ec140e7e8112"
    },
    "runtime_environment": "spark-2.0",
    "deployments": {
      "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models/23d93142-c740-4cec-9cd6-18dad8bb9717/deployments",
      "count": 0
    },
    "training_data_schema": {
      "type": "struct",
      "fields": [
        {
  

#### Get all models

In [29]:
models_details = client.repository.get_details()
print(json.dumps(models_details, indent=2))

{
  "resources": [
    {
      "metadata": {
        "created_at": "2017-10-10T11:26:32.270Z",
        "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models/23d93142-c740-4cec-9cd6-18dad8bb9717",
        "modified_at": "2017-10-10T11:26:32.536Z",
        "guid": "23d93142-c740-4cec-9cd6-18dad8bb9717"
      },
      "entity": {
        "label_col": "PRODUCT_LINE",
        "latest_version": {
          "created_at": "2017-10-10T11:26:32.536Z",
          "url": "https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/23d93142-c740-4cec-9cd6-18dad8bb9717/versions/57899b60-5336-448d-81cb-ec140e7e8112",
          "guid": "57899b60-5336-448d-81cb-ec140e7e8112"
        },
        "runtime_environment": "spark-2.0",
        "deployments": {
          "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models/23d93142-c740-4cec-9cd6-18dad8bb9717/deployments",
          "count": 0


### 4.3: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [30]:
loaded_model = client.repository.load(published_model.uid)

You can pass test_data to loaded model ``transform()`` method to make sure that model has been loaded correctly.

In [31]:
test_predictions = loaded_model.transform(test_data)

In [32]:
test_predictions.show(3)

+-----------------+------+---+--------------+-----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|     PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS| PROFESSION|label|PROFESSION_IX|GENDER_IX|MARITAL_STATUS_IX|          features|       rawPrediction|         probability|prediction|      predictedLabel|
+-----------------+------+---+--------------+-----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|Camping Equipment|     F| 18|        Single|      Other|  0.0|          0.0|      1.0|              1.0|[1.0,18.0,1.0,0.0]|[4.94415029685990...|[0.24720751484299...|       1.0|Personal Accessories|
|Camping Equipment|     F| 18|        Single|     Retail|  0.0|          7.0|      1.0|              1.0|[1.0,18.0,1.0,7.0]|[2.88119976249765...|[0.14405998812488...|       1.0|Personal Accessories|
|Camp

As you can see the loaded model works. You have already learned how save and load the model from Watson Machine Learning repository.

### 4.4: Delete model

You can delete published model from Watson Machine Learning repository using below code. The code is commented out at this stage since the model will be needed later for deployment.

<a id="scoring"></a>
## 5. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning Client. 

### 5.1: Create online deployment

In [33]:
scoring_endpoint = client.deployments.create(published_model.uid, "Deployment of locally created model XXX")

Now you can print an online scoring endpoint. 

In [34]:
print(scoring_endpoint)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models/23d93142-c740-4cec-9cd6-18dad8bb9717/deployments/f26b7e73-a342-491e-9339-51b48abae624/online


### 5.2: Get deployments

In [35]:
deployments = client.deployments.get_details()

In [36]:
print(json.dumps(deployments, indent=2))

{
  "resources": [
    {
      "metadata": {
        "created_at": "2017-09-29T14:38:32.181Z",
        "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models/77022504-8ad9-45c6-9b25-f51acd0e5222/deployments/b83dfdf8-db9a-4d38-b096-d8b91fd011ae",
        "modified_at": "2017-09-29T14:44:02.095Z",
        "guid": "b83dfdf8-db9a-4d38-b096-d8b91fd011ae"
      },
      "entity": {
        "scoring_url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models/77022504-8ad9-45c6-9b25-f51acd0e5222/deployments/b83dfdf8-db9a-4d38-b096-d8b91fd011ae/online",
        "runtime_environment": "spark-2.0",
        "published_model": {
          "created_at": "2017-09-28T13:14:02.899Z",
          "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models/77022504-8ad9-45c6-9b25-f51acd0e5222",
          "guid": "77022504-8ad9-45c6-9b25-f51

You can get deployment_url by parsing deployment details.

In [37]:
deployment_url = deployments.get('resources')[0].get('metadata').get('url')
print(deployment_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models/77022504-8ad9-45c6-9b25-f51acd0e5222/deployments/b83dfdf8-db9a-4d38-b096-d8b91fd011ae


### 5.3: Score

You can use below method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [38]:
scoring_payload = {"fields": ["GENDER","AGE","MARITAL_STATUS","PROFESSION"],"values": [["M",23,"Single","Student"],["M",55,"Single","Executive"]]}

Use ``client.deployments.score()`` method to run scoring.

In [39]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)

In [40]:
print(json.dumps(predictions, indent=2))

{
  "fields": [
    "GENDER",
    "AGE",
    "MARITAL_STATUS",
    "PROFESSION",
    "PRODUCT_LINE",
    "label",
    "PROFESSION_IX",
    "GENDER_IX",
    "MARITAL_STATUS_IX",
    "features",
    "rawPrediction",
    "probability",
    "prediction",
    "predictedLabel"
  ],
  "values": [
    [
      "M",
      23,
      "Single",
      "Student",
      "Camping Equipment",
      0.0,
      6.0,
      0.0,
      1.0,
      [
        0.0,
        23.0,
        1.0,
        6.0
      ],
      [
        4.5146661875758465,
        7.57017273688273,
        5.662967039590646,
        0.1716508249665755,
        2.0805432109842013
      ],
      [
        0.22573330937879232,
        0.3785086368441365,
        0.2831483519795323,
        0.008582541248328775,
        0.10402716054921006
      ],
      1.0,
      "Personal Accessories"
    ],
    [
      "M",
      55,
      "Single",
      "Executive",
      "Camping Equipment",
      0.0,
      3.0,
      0.0,
      1.0,
      [
        

### 5.4: Delete deployment

Use the following method to delete deployment.

In [41]:
client.deployments.delete(deployment_url)

You can check number of depoyments by running ``client.deployment.get()`` method.

In [42]:
client.deployments.get_details()

{'count': 1,
 'resources': [{'entity': {'deployed_version': {'created_at': '2017-10-10T11:26:32.536Z',
     'guid': '57899b60-5336-448d-81cb-ec140e7e8112',
     'url': 'https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/23d93142-c740-4cec-9cd6-18dad8bb9717/versions/57899b60-5336-448d-81cb-ec140e7e8112'},
    'description': 'Model deployment',
    'model_type': 'sparkml-model-2.0',
    'name': 'Deployment of locally created model XXX',
    'published_model': {'author': {},
     'created_at': '2017-10-10T11:27:06.975Z',
     'guid': '23d93142-c740-4cec-9cd6-18dad8bb9717',
     'name': 'LOCALLY created Product Line Prediction model',
     'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models/23d93142-c740-4cec-9cd6-18dad8bb9717'},
    'runtime_environment': 'spark-2.0',
    'scoring_url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models/23d93142-c740-4cec-9cd6-18dad

### 5.5: Delete model

In [43]:
client.repository.delete(published_model.uid)

You can check number of your models by running below ``client.repository.get_details()`` method.

In [44]:
client.repository.get_details()

{'count': 5,
 'resources': [{'entity': {'author': {},
    'deployments': {'count': 0,
     'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/81eb122f-a170-4efd-86b6-4a8afa16d30b/published_models/4285837c-6a5a-40c7-8d37-e63f75b05182/deployments'},
    'input_data_schema': {'fields': [{'metadata': {},
       'name': 'GENDER',
       'nullable': True,
       'type': 'string'},
      {'metadata': {}, 'name': 'AGE', 'nullable': True, 'type': 'integer'},
      {'metadata': {},
       'name': 'MARITAL_STATUS',
       'nullable': True,
       'type': 'string'},
      {'metadata': {},
       'name': 'PROFESSION',
       'nullable': True,
       'type': 'string'}],
     'type': 'struct'},
    'label_col': 'PRODUCT_LINE',
    'model_type': 'sparkml-model-2.0',
    'name': 'LOCALLY created Product Line Prediction model',
    'runtime_environment': 'spark-2.0',
    'training_data_schema': {'fields': [{'metadata': {},
       'name': 'PRODUCT_LINE',
       'nullable': True,
       'type': 

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/pm_service_api_spark.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.